In [12]:
# Feature Engineering + Discovery Engine with custom features
import pandas as pd
import numpy as np
import random
import warnings
import functools
import os
from scipy.stats import linregress
import itertools # NEW: For combinatorial setup generation

warnings.filterwarnings('ignore')

# --- DEFINITIONS AND CONFIGURATION ---
# Define the explicit list of tradable tickers (full sheet names from your All_Tickers copy.xlsx)
# These will be the only assets for which returns are calculated for strategy performance.
TRADABLE_TICKERS = [
    'QQQ US Equity', 'SPY US Equity', 'XLK US Equity', 'XLF US Equity',
    'XLE US Equity', 'ARKK US Equity', 'VIX Index', 'GLD US Equity',
    'NBIS US Equity', 'LLY US Equity', 'TSLA US Equity', 'AAPL US Equity',
    'NVDA US Equity'
    # DXY Curncy, USGG10YR Index, CO1 Comdty are explicitly excluded from tradable targets
    # as per your request (due to no options data for future simulation, etc.)
]

# Define the file paths
MAIN_DATA_FILE = 'All_Tickers copy.xlsx'
MACRO_DATA_FILE = 'Macro_tickers_no_nan_cols.xlsx'

# Setup Generation Configuration
SETUP_LENGTHS_TO_EXPLORE = [2, 3, 4] # Explore setups with 2, 3, and 4 conditions
MIN_INITIAL_SUPPORT_FILTER = 10 # Minimum number of trigger days for a setup to be considered

# Option Simulation Configuration
OPTION_SIM_HORIZON_DAYS = 10 # Days to expiration for simulated options
RISK_FREE_RATE = 0.01 # Annual risk-free rate for option premium estimation

# --- END DEFINITIONS AND CONFIGURATION ---


print('Loading raw workbooks …')
raw = None # Initialize raw to None

# --- Custom Data Loading Function to handle sheet names as prefixes ---
def load_and_merge_excel(file_path, existing_df=None):
    """Loads an Excel file, prepends sheet names to columns (except Date), and merges."""
    try:
        xls = pd.ExcelFile(file_path)
        current_df = existing_df.copy() if existing_df is not None else None

        for sh_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sh_name)
            # Prepend sheet name to all columns except 'Date'
            df.columns = [f"{sh_name}_{col}" if col != 'Date' else col for col in df.columns]

            if current_df is None:
                current_df = df
            else:
                # Use outer merge to keep all dates from all sheets
                current_df = current_df.merge(df, on='Date', how='outer')
        return current_df
    except FileNotFoundError:
        print(f"Error: '{file_path}' not found. Please ensure the file is in the correct directory.")
        return existing_df # Return existing_df or None if first file
    except Exception as e:
        print(f"An unexpected error occurred during Excel loading of '{file_path}': {e}")
        return existing_df # Return existing_df or None if first file

# Load main data
raw = load_and_merge_excel(MAIN_DATA_FILE)

# Load macro data if main data was loaded successfully
if raw is not None and not raw.empty:
    raw = load_and_merge_excel(MACRO_DATA_FILE, existing_df=raw)
else:
    print("Main data could not be loaded, skipping macro data loading.")
    raw = pd.DataFrame({'Date': pd.to_datetime([])}) # Fallback to empty DF if main load failed

# Final cleaning and sorting
if raw is not None and not raw.empty:
    raw = raw.sort_values('Date').reset_index(drop=True)
    raw.fillna(method='ffill', inplace=True)
else:
    print("No data loaded. Raw DataFrame is empty.")
    raw = pd.DataFrame({'Date': pd.to_datetime([])}) # Ensure raw is a DataFrame

print('Raw shape:', raw.shape)
print('Example columns after loading:')
print(raw.columns[:5].tolist()) # Print first 5 columns to verify naming


# --- NEW ADDITION (Phase 1, Item 1): Dynamic Ticker Identification Refined ---
# all_tickers will now refer to ALL unique ticker prefixes found in columns after loading,
# not just PX_LAST, but it will exclude 'Date' and common feature parts.
# This is mainly for feature engineering loops.
all_column_prefixes = sorted(list(set([c.split('_')[0] for c in raw.columns if '_' in c and c != 'Date'])))
# Filter out common feature names that might accidentally be parsed as tickers
COMMON_FEATURE_PREFIXES = ['Last', 'Open', 'High', 'Low', 'VWAP', 'Volume', 'IVOL', 'Implied', 'Total',
                           '30', '10', '60', 'Hist.', '1st', 'Put', 'Dates', 'CHG', 'FFA', 'INJCJC',
                           'NFP', 'JOBS', 'CPI', 'CTII10', 'LF94TRUU', 'SPX', 'USSW10', 'MLCX3CRT',
                           'FARBAST', 'BSPGCPUS', 'SPCSUSA', 'SPCS20SM', 'CONSSENT']
actual_ticker_prefixes = [p for p in all_column_prefixes if p not in COMMON_FEATURE_PREFIXES]
# Ensure that TRADABLE_TICKERS are definitely included even if they don't have a specific prefixed column.
# This list is used for generalizing feature engineering.
all_tickers = sorted(list(set(TRADABLE_TICKERS + actual_ticker_prefixes)))

print(f'\nIdentified all relevant prefixes/tickers for feature engineering: {all_tickers}')
print(f'Actual tradable tickers for returns: {TRADABLE_TICKERS}')


# --- Revised Helper functions ---
def first_col_containing(ticker_full_name, substr=''):
    """
    Finds the first column name in raw that matches the pattern 'ticker_full_name_substr'.
    Handles cases where substr might be 'PX_LAST' and the actual column name is 'ticker_full_name_Last_Price_PX_LAST'.
    """
    # Prefer explicit matches for PX_LAST which can have two forms
    if substr == 'PX_LAST':
        potential_col_name_long_price = f"{ticker_full_name}_Last_Price_PX_LAST"
        if potential_col_name_long_price in raw.columns:
            return potential_col_name_long_price

        # Check for the simpler form if the above doesn't exist
        potential_col_name_short_px = f"{ticker_full_name}_PX_LAST"
        if potential_col_name_short_px in raw.columns:
            return potential_col_name_short_px

    # Generic search for other substrings
    for c in raw.columns:
        # Check for 'FULL_TICKER_NAME_SUBSTR' (e.g., QQQ US Equity_VOLUME)
        if c.startswith(ticker_full_name) and substr in c:
            return c
    return None

def safe_series(col_name):
    """Returns a column as a Series, or an empty Series if column does not exist."""
    return raw[col_name] if col_name and col_name in raw.columns else pd.Series(index=raw.index, dtype=float)

# --- NEW ADDITION (Phase 1, Item 2): Helper function for Block Bootstrap Sharpe ---
def block_bootstrap_sharpe(returns_series, block_size, num_iterations=1000, annualize=True, trading_days_per_year=252):
    """
    Calculates the Sharpe Ratio using block bootstrapping to account for serial correlation.
    """
    returns_series = returns_series.dropna()
    if len(returns_series) < block_size or len(returns_series) < 2:
        return 0.0, 0.0, 0.0

    sharpes = []
    blocks = []
    for i in range(0, len(returns_series), block_size):
        block = returns_series.iloc[i : i + block_size]
        if not block.empty:
            blocks.append(block)

    if not blocks:
        return 0.0, 0.0, 0.0

    n_blocks_to_sample = int(np.ceil(len(returns_series) / block_size))

    for _ in range(num_iterations):
        resampled_returns_list = []
        sampled_blocks_indices = np.random.choice(len(blocks), n_blocks_to_sample, replace=True)
        for idx in sampled_blocks_indices:
            resampled_returns_list.append(blocks[idx])

        resampled_returns = pd.concat(resampled_returns_list).iloc[:len(returns_series)]

        if resampled_returns.std() > 1e-9: # Original threshold for non-zero std
            daily_sharpe = resampled_returns.mean() / resampled_returns.std()
            if annualize:
                sharpes.append(daily_sharpe * np.sqrt(trading_days_per_year))
            else:
                sharpes.append(daily_sharpe)
        else:
            sharpes.append(0.0)

    sharpes_sorted = sorted(sharpes)
    if not sharpes_sorted:
        return 0.0, 0.0, 0.0

    median_sharpe = np.median(sharpes_sorted)
    lower_ci = sharpes_sorted[int(0.05 * num_iterations)]
    upper_ci = sharpes_sorted[int(0.95 * num_iterations)]

    return median_sharpe, lower_ci, upper_ci
# --- END NEW ADDITION ---


# --- NEW ADDITION (Phase 2, Item 1): Option Payoff Simulation Helpers ---
# Simplified Black-Scholes premium estimation (for ATM options)
def estimate_atm_premium(S, ivol, T_days, r_annual=RISK_FREE_RATE, option_type='call'):
    """
    Estimates ATM option premium using a simplified Black-Scholes like approach.
    This is a rough proxy, not a full BSM model.
    """
    if S <= 0 or ivol <= 0 or T_days <= 0:
        return 0.001 # Return a minimal premium if inputs are invalid

    T_years = T_days / 252.0 # Annualize time to expiration

    # A common rule of thumb for ATM options: premium is roughly 0.4 * S * IVOL * sqrt(T_years)
    # The 0.4 factor approximates N(d1) and N(d2) for ATM options around 0.5.
    premium_estimate = 0.4 * S * ivol * np.sqrt(T_years)

    return max(premium_estimate, 0.001) # Ensure premium is at least minimal

def simulate_option_pnl(current_price, future_price, ivol_at_entry, horizon_days, entry_direction):
    """
    Simulates PnL for buying a simple ATM call or put.
    """
    if pd.isna(current_price) or pd.isna(future_price) or pd.isna(ivol_at_entry):
        return np.nan

    strike = current_price # At-The-Money option

    if entry_direction == 'long':
        # Simulate buying an ATM Call
        premium = estimate_atm_premium(current_price, ivol_at_entry, horizon_days, option_type='call')
        payoff = max(future_price - strike, 0)
        pnl = payoff - premium
    elif entry_direction == 'short':
        # Simulate buying an ATM Put (for a 'short' signal in underlying)
        premium = estimate_atm_premium(current_price, ivol_at_entry, horizon_days, option_type='put')
        payoff = max(strike - future_price, 0)
        pnl = payoff - premium
    else: # Mixed or other, no specific option trade
        pnl = np.nan # Not applicable

    return pnl
# --- END Option Payoff Simulation Helpers ---


print('\nEngineering custom features …')
feat = pd.DataFrame({'Date': raw['Date']})

# --- NEW ADDITION: Fractional Differencing Helper (moved here for feature engineering use) ---
def frac_diff(series, d=0.5, window='full'):
    """
    Computes fractionally differenced series.
    Adapted from Lopez de Prado's "Advances in Financial Machine Learning".
    """
    if not isinstance(series, pd.Series):
        series = pd.Series(series)

    # Compute weights
    weights = [1.]
    for k in range(1, series.shape[0]):
        weights.append(-weights[-1] * (d - k + 1) / k)
    weights = np.array(weights[::-1]) # Reverse for convolution

    output = pd.Series(index=series.index, dtype=float)
    for i in range(series.shape[0]):
        if window == 'full':
            start = 0
        else: # fixed window
            start = max(0, i - window + 1)

        subset = series.iloc[start : i + 1]

        # Ensure weights are aligned with subset length
        current_weights = weights[-(i - start + 1):]
        if len(subset) == len(current_weights):
            output.iloc[i] = np.dot(current_weights, subset)
    return output.dropna()
# --- END Fractional Differencing Helper ---


# 1. VOLATILITY-BASED FEATURES
print("  - Volatility-based features")
# Use TRADABLE_TICKERS to ensure we only process assets with full options data where relevant
for ticker in TRADABLE_TICKERS:
    # Get relevant IVOL columns for the ticker
    ivol_10d_col = first_col_containing(ticker, '10_Day_Call_Implied_Volatility_CALL_IMP_VOL_10D')
    ivol_30d_col = first_col_containing(ticker, '30_Day_Call_Implied_Volatility_CALL_IMP_VOL_30D')
    ivol_60d_col = first_col_containing(ticker, '60_Day_Call_Implied_Volatility_CALL_IMP_VOL_60D')

    # NEW ADVANCED FEATURE: Implied Volatility Term Structure Slope
    if ivol_60d_col and ivol_10d_col:
        feat[f'{ticker}_IVOL_Term_Structure_Slope'] = safe_series(ivol_60d_col) - safe_series(ivol_10d_col)

    # NEW ADVANCED FEATURE: Implied Volatility Skew (Approximation)
    # Using 1M Call 40D and 1M Put 50D as proxies for OTM Call and ATM Put
    call_40d_ivol_col = first_col_containing(ticker, '1st_Month_Call_Imp_Vol_40_Delta_LIVE_1M_CALL_IMP_VOL_40DELTA_DFLT')
    put_50d_ivol_col = first_col_containing(ticker, '1st_Month_Put_Imp_Vol_50_Delta_LIVE_1M_PUT_IMP_VOL_50DELTA_DFLT')
    if call_40d_ivol_col and put_50d_ivol_col:
        feat[f'{ticker}_IVOL_Skew_Approx'] = safe_series(put_50d_ivol_col) - safe_series(call_40d_ivol_col)


    # Original IVOL-based features, adapted to new naming
    # Loop through specific IVOL types for this ticker
    specific_ivol_suffixes = ['IVOL_SIGMA', 'IVOL_DELTA', 'IVOL_MONEYNESS', 'CALL_IMP_VOL_30D', 'CALL_IMP_VOL_10D', 'CALL_IMP_VOL_60D', 'PUT_IMP_VOL_30D', 'PUT_IMP_VOL_10D', 'PUT_IMP_VOL_60D']
    for ivol_suffix in specific_ivol_suffixes:
        col_name = first_col_containing(ticker, ivol_suffix)
        if col_name:
            # Original slope logic (if relevant to this specific IVOL column and its '10D' counterpart)
            # This logic might need further refinement based on exact suffix patterns (e.g., '60_Day_Call_...' vs '10_Day_Call_...')
            if '60_Day_Call_Implied_Volatility_CALL_IMP_VOL_60D' in col_name and \
               first_col_containing(ticker, '10_Day_Call_Implied_Volatility_CALL_IMP_VOL_10D'):
                feat[f'{ticker}_IVOL_Call_Slope'] = safe_series(col_name) - \
                                                    safe_series(first_col_containing(ticker, '10_Day_Call_Implied_Volatility_CALL_IMP_VOL_10D'))

            diff = safe_series(col_name).diff()
            z = (diff - diff.rolling(30).mean()) / diff.rolling(30).std()
            feat[col_name+'_shock'] = (z > 2).astype(int)

            vol_col_name = first_col_containing(ticker, 'VOLUME') # Use helper to find generic VOLUME

            if vol_col_name:
                feat[col_name+'_div'] = safe_series(col_name) / safe_series(vol_col_name)
            else:
                feat[col_name+'_div'] = np.nan # Assign NaN if volume not found for division


# 2. DERIV FLOW & SENTIMENT
print("  - Deriv Flow & Sentiment features")
# Iterate through all_tickers (tradable + macro factors that might have these features)
for ticker in all_tickers:
    pc_col = first_col_containing(ticker, 'PUT_CALL_VOLUME_RATIO_CUR_DAY')
    if pc_col:
        feat[pc_col+'_ema5']=safe_series(pc_col).ewm(span=5,adjust=False).mean()

    open_int_col = first_col_containing(ticker, 'OPEN_INT_TOTAL_CALL')
    if open_int_col:
        feat[open_int_col+'_chg3']=safe_series(open_int_col).pct_change(3)

    # Only process real volume columns (not VWAP or other derived volumes)
    volm_col = first_col_containing(ticker, 'Volume_-_Realtime_VOLUME')
    if volm_col:
        feat[volm_col+'_z']=(safe_series(volm_col)-safe_series(volm_col).rolling(30).mean())/safe_series(volm_col).rolling(30).std()

# Generalize smart_money_flag: loop through dynamically identified all_tickers
for ticker in all_tickers:
    open_int_t_col = first_col_containing(ticker, 'OPEN_INT_TOTAL_CALL')
    ivol_10d_col = first_col_containing(ticker, '10_Day_Call_Implied_Volatility_CALL_IMP_VOL_10D')

    if open_int_t_col and ivol_10d_col:
        feat[f'{ticker}_smart_money_flag'] = ((safe_series(open_int_t_col).pct_change() > 0) & \
                                              (safe_series(ivol_10d_col).pct_change() > 0)).astype(int)

# 3. CROSS-ASSET CORRELATIONS
print("  - Cross-Asset Correlations")
# Use the TRADABLE_TICKERS for robust pair selection, and also include macro factors if their PX_LAST is available
# TRADABLE_TICKERS and potentially key macro factors (DXY, USGG10YR, SPX)
correlation_universe = sorted(list(set(TRADABLE_TICKERS + [t for t in all_tickers if t in ['DXY Curncy', 'USGG10YR Index', 'SPX Index', 'CO1 Comdty', 'USGG2YR Index']])))

# Form pairs dynamically or with a predefined list for efficiency (all-pairs can be very large)
# Let's use a combination of predefined and dynamically generated pairs for flexibility
dynamic_pairs = []
# All tradable pairs (excluding self-correlation and duplicates)
for i in range(len(TRADABLE_TICKERS)):
    for j in range(i + 1, len(TRADABLE_TICKERS)):
        dynamic_pairs.append((TRADABLE_TICKERS[i], TRADABLE_TICKERS[j]))

# Add specific cross-asset pairs involving macro factors if desired
dynamic_pairs.extend([
    ('SPY US Equity', 'VIX Index'), # Already in tradable, but useful example
    ('SPY US Equity', 'USGG10YR Index'),
    ('SPY US Equity', 'DXY Curncy'),
    ('SPY US Equity', 'CO1 Comdty'),
    ('SPY US Equity', 'USGG2YR Index'), # New macro correlation
    ('VIX Index', 'USGG10YR Index'),
    ('VIX Index', 'DXY Curncy'),
    ('VIX Index', 'CO1 Comdty'),
])
# Remove duplicates if any
dynamic_pairs = list(set(dynamic_pairs))

for t1,t2 in dynamic_pairs:
    p1=first_col_containing(t1,'PX_LAST'); p2=first_col_containing(t2,'PX_LAST')
    if p1 and p2:
        s1=safe_series(p1); s2=safe_series(p2)
        # Ensure series are not empty after safe_series and can be aligned
        if not s1.empty and not s2.empty:
            aligned_data = pd.DataFrame({'s1': s1, 's2': s2}).dropna()
            if len(aligned_data) > 60: # Need enough data for rolling correlations
                c20=aligned_data['s1'].rolling(20).corr(aligned_data['s2'])
                c60=aligned_data['s1'].rolling(60).corr(aligned_data['s2'])
                feat[f'{t1}_{t2}_c20']=c20
                feat[f'{t1}_{t2}_c60']=c60
                feat[f'{t1}_{t2}_cZ']=(c20-c20.rolling(60).mean())/c20.rolling(60).std()
                feat[f'{t1}_{t2}_cDelta']=c20-c60

                # NEW ADVANCED FEATURE: Rolling Beta
                # Calculate daily returns for beta
                ret1 = s1.pct_change().dropna()
                ret2 = s2.pct_change().dropna()

                # Align indices for beta calculation
                aligned_returns = pd.DataFrame({'ret1': ret1, 'ret2': ret2}).dropna()

                if not aligned_returns.empty and aligned_returns['ret2'].var() != 0:
                    rolling_beta = aligned_returns['ret1'].rolling(window=60).cov(aligned_returns['ret2']) / \
                                   aligned_returns['ret2'].rolling(window=60).var()
                    feat[f'{t1}_{t2}_rolling_beta'] = rolling_beta


# 4. MACRO TRIGGERS (NOW INCLUDING NEW BLOOMBERG MACRO DATA)
print("  - Macro Trigger features")

# Core Macro Index PX_LAST lookups (using exact names from your files)
dxy_px=first_col_containing('DXY Curncy','PX_LAST')
ust10_px=first_col_containing('USGG10YR Index','PX_LAST')
spy_px=first_col_containing('SPY US Equity','PX_LAST')
vix_px=first_col_containing('VIX Index','PX_LAST')

feat['MPI']=(safe_series(dxy_px).pct_change().rolling(3).sum()+safe_series(ust10_px).pct_change().rolling(3).sum()).shift(0)
feat['VIX_gt20']=(safe_series(vix_px)>20).astype(int)
feat['DXY_rising']=(safe_series(dxy_px).pct_change()>0).astype(int)
feat['SPY_below_MA20']=(safe_series(spy_px)<safe_series(spy_px).rolling(20).mean()).astype(int)
feat['fear_overdrive']=((feat['VIX_gt20']==1)&(feat['DXY_rising']==1)&(feat['SPY_below_MA20']==1)).astype(int)

xlk_px=first_col_containing('XLK US Equity','PX_LAST'); xle_px=first_col_containing('XLE US Equity','PX_LAST')
feat['sector_rotation']=safe_series(xlk_px).pct_change(5)-safe_series(xle_px).pct_change(5)

# --- NEW MACRO FEATURES from Macro_tickers_no_nan_cols.xlsx ---
# Yield Curve: USGG10YR vs USGG2YR spread
ust2_px = first_col_containing('USGG2YR Index', 'PX_LAST')
if ust10_px and ust2_px:
    feat['UST10Y_2Y_Spread'] = safe_series(ust10_px) - safe_series(ust2_px)
    feat['UST10Y_2Y_Spread_chg'] = feat['UST10Y_2Y_Spread'].pct_change()

# Inflation
cpi_yoy_px = first_col_containing('CPI YOY Index', 'PX_LAST')
cpi_chng_px = first_col_containing('CPI CHNG Index', 'PX_LAST')
if cpi_yoy_px:
    feat['CPI_YOY_mom3'] = safe_series(cpi_yoy_px).pct_change(3)
    feat['CPI_YOY_z'] = (safe_series(cpi_yoy_px) - safe_series(cpi_yoy_px).rolling(12).mean()) / safe_series(cpi_yoy_px).rolling(12).std()
if cpi_chng_px:
    feat['CPI_CHNG_mom3'] = safe_series(cpi_chng_px).pct_change(3)

# Employment Data
injcjc_px = first_col_containing('INJCJC Index', 'PX_LAST')
nfp_tch_px = first_col_containing('NFP TCH Index', 'PX_LAST')
jobs_us_equity_px = first_col_containing('JOBS US Equity', 'PX_LAST') # Treated as macro factor
if injcjc_px:
    feat['INJCJC_shock'] = (safe_series(injcjc_px).diff() > safe_series(injcjc_px).diff().rolling(20).std() * 2).astype(int)
if nfp_tch_px:
    feat['NFP_TCH_mom3'] = safe_series(nfp_tch_px).pct_change(3)
if jobs_us_equity_px:
    feat['JOBS_US_Equity_mom3'] = safe_series(jobs_us_equity_px).pct_change(3)

# Other Macro / Credit / Rates
ffa_comdty_px = first_col_containing('FFA Comdty', 'PX_LAST')
ctii10_govt_px = first_col_containing('CTII10 Govt', 'PX_LAST')
ussw10_curncy_px = first_col_containing('USSW10 Curncy', 'PX_LAST')
mlcx3crt_index_px = first_col_containing('MLCX3CRT Index', 'PX_LAST')
farbast_index_px = first_col_containing('FARBAST Index', 'PX_LAST')
bspgcpus_index_px = first_col_containing('BSPGCPUS Index', 'PX_LAST')
spcsusa_index_px = first_col_containing('SPCSUSA Index', 'PX_LAST')
spcs20sm_index_px = first_col_containing('SPCS20SM Index', 'PX_LAST')
conssent_index_px = first_col_containing('CONSSENT Index', 'PX_LAST')
lf94truu_index_vol30d = first_col_containing('LF94TRUU Index', 'VOLATILITY_30D')


if ffa_comdty_px: feat['FFA_Spread'] = safe_series(ffa_comdty_px) - safe_series(ust2_px) # Example spread, needs ust2_px
if ctii10_govt_px: feat['CTII10_mom'] = safe_series(ctii10_govt_px).pct_change()
if ussw10_curncy_px: feat['USSW10_chg'] = safe_series(ussw10_curncy_px).pct_change()
if mlcx3crt_index_px: feat['MLCX3CRT_chg'] = safe_series(mlcx3crt_index_px).pct_change()
if farbast_index_px: feat['FARBAST_mom'] = safe_series(farbast_index_px).pct_change()
if bspgcpus_index_px: feat['BSPGCPUS_mom'] = safe_series(bspgcpus_index_px).pct_change()
if spcsusa_index_px: feat['SPCSUSA_mom'] = safe_series(spcsusa_index_px).pct_change()
if spcs20sm_index_px: feat['SPCS20SM_mom'] = safe_series(spcs20sm_index_px).pct_change()
if conssent_index_px: feat['CONSSENT_mom'] = safe_series(conssent_index_px).pct_change()
if lf94truu_index_vol30d: feat['LF94TRUU_Vol_Signal'] = safe_series(lf94truu_index_vol30d) / safe_series(lf94truu_index_vol30d).rolling(60).mean()


# 5. MOMENTUM / VOL FRACTALS
print("  - Momentum/Vol Fractals")
for tk in all_tickers: # Loop through all prefixes/tickers
    px_col=first_col_containing(tk,'PX_LAST')
    if not px_col: continue
    px=safe_series(px_col)

    mom5=px.pct_change(5)
    vol20=px.pct_change().rolling(20).std()
    feat[tk+'_mom5_vol20']=mom5/vol20
    ma=px.rolling(20).mean(); std=px.rolling(20).std()
    feat[tk+'_pctB']=(px-(ma-2*std))/(4*std)

    # NEW ADVANCED FEATURE: Fractional Differencing
    # Applying fractional differencing to the primary price series of each ticker
    if not px.empty and len(px) > 100: # Ensure enough data points for fractional differencing
        try:
            feat[f'{tk}_frac_diff_0_5'] = frac_diff(px, d=0.5, window=100) # Window for efficiency
        except Exception as e:
            print(f"Warning: Could not compute fractional differencing for {tk}: {e}")
            feat[f'{tk}_frac_diff_0_5'] = np.nan # Assign NaN on error


# 6. Shift +1 day and merge basic returns
feat=feat.shift(1) # Shift all engineered features

panel = feat.copy() # Start panel with shifted features

# Prepare returns for evaluation based *only* on TRADABLE_TICKERS
price_cols_for_returns = []
for ticker_full_name in TRADABLE_TICKERS:
    px_col_name = first_col_containing(ticker_full_name, 'PX_LAST')
    if px_col_name:
        price_cols_for_returns.append(px_col_name)
    else:
        print(f"Warning: PX_LAST column not found for tradable ticker '{ticker_full_name}'. It will be excluded from return calculations.")

prices=raw[['Date']+price_cols_for_returns].copy(); prices.set_index('Date',inplace=True)
returns={h:prices.pct_change(h).shift(-h) for h in [1,3,5,10,21]}


# Merge tradable returns into panel
for tk_full_name in TRADABLE_TICKERS:
    px_col_name = first_col_containing(tk_full_name, 'PX_LAST')
    if px_col_name and px_col_name in raw.columns: # Ensure the column exists in raw
        # Calculate return and rename it consistently with the full ticker name
        ret_series = raw[px_col_name].pct_change().shift(1).rename(f'{tk_full_name}_ret1')
        panel = pd.concat([panel, ret_series], axis=1)

panel.index=raw['Date']
print('Engineered panel shape:',panel.shape)


# Generate primitive signals
print('\nBuilding primitive signals from engineered panel …')
signals={}
# --- NEW ADDITION (Phase 2, Item 2): Helper functions for inferring signal direction and type ---
def get_primitive_direction(primitive_name):
    """Infers the directional bias (+1 for long, -1 for short, 0 for neutral) of a primitive signal."""
    if '>80' in primitive_name or 'z>1.5' in primitive_name or 'ma5>ma20' in primitive_name or 'rising' in primitive_name:
        return 1 # Long bias
    elif '<20' in primitive_name or 'z<-1.name' in primitive_name or 'ma5<ma20' in primitive_name or 'below_MA' in primitive_name:
        return -1 # Short bias
    elif 'shock' in primitive_name and '_shock' in primitive_name: # Shocks are generally neutral directionally on their own
        return 0
    elif 'slope' in primitive_name and '_slope' in primitive_name: # Slopes could be directional, but often neutral depending on context
        return 0
    elif 'Spread' in primitive_name and '_Spread' in primitive_name: # Spreads can be directional, depending on interpretation
        return 0
    elif 'Vol_Signal' in primitive_name: # Volatility signals are often neutral
        return 0
    return 0 # Default to neutral if no clear directional bias

def get_primitive_signal_type(primitive_name):
    """Infers the broad category/type of a primitive signal based on its name."""
    # Order matters: more specific keywords first
    if 'IVOL_Term_Structure_Slope' in primitive_name or 'IVOL_Skew_Approx' in primitive_name or 'IVOL' in primitive_name or 'VIX' in primitive_name or 'vol' in primitive_name or '_shock' in primitive_name:
        return 'volatility'
    elif 'mom' in primitive_name or 'pctB' in primitive_name or '_chg' in primitive_name or '_ret' in primitive_name or 'rising' in primitive_name:
        return 'momentum'
    elif '_c20' in primitive_name or '_c60' in primitive_name or '_cZ' in primitive_name or '_cDelta' in primitive_name or '_beta' in primitive_name:
        return 'correlation'
    elif 'DXY' in primitive_name or 'USGG' in primitive_name or 'MPI' in primitive_name or 'fear_overdrive' in primitive_name or 'CPI' in primitive_name or 'INJCJC' in primitive_name or 'NFP' in primitive_name or 'JOBS' in primitive_name or 'FFA' in primitive_name or 'CTII10' in primitive_name or 'USSW10' in primitive_name or 'MLCX3CRT' in primitive_name or 'FARBAST' in primitive_name or 'BSPGCPUS' in primitive_name or 'SPCSUSA' in primitive_name or 'SPCS20SM' in primitive_name or 'CONSSENT' in primitive_name or 'LF94TRUU' in primitive_name:
        return 'macro'
    elif 'PUT_CALL_VOLUME_RATIO' in primitive_name or 'smart_money_flag' in primitive_name or 'Short_Interest_Ratio' in primitive_name:
        return 'sentiment'
    elif 'VOLUME' in primitive_name:
        return 'volume'
    elif 'OPEN_INT' in primitive_name:
        return 'open_interest'
    elif 'frac_diff' in primitive_name:
        return 'fractional_differencing' # New category for this advanced feature
    return 'other'
# --- END NEW ADDITION ---

for col in panel.columns.drop('Date',errors='ignore'):
    s=panel[col]

    # Ensure the series is numeric before trying rank, mean, std
    if pd.api.types.is_numeric_dtype(s):
        s_clean = s.replace([np.inf, -np.inf], np.nan).dropna()

        if s_clean.empty or s_clean.std() == 0:
            continue

        rank=s_clean.rank(pct=True)
        signals[col+'>80']=rank>0.8
        signals[col+'<20']=rank<0.2

        rolling_std_60 = s_clean.rolling(60).std()
        valid_std_mask = rolling_std_60 > 1e-9

        z = pd.Series(np.nan, index=s_clean.index, dtype=float)
        z[valid_std_mask] = (s_clean - s_clean.rolling(60).mean())[valid_std_mask] / rolling_std_60[valid_std_mask]

        signals[col+'_z>1.5']=z>1.5
        signals[col+'_z<-1.5']=z<-1.5

        ma5=s_clean.rolling(5).mean(); ma20=s_clean.rolling(20).mean()
        signals[col+'_ma5>ma20']=ma5>ma20
print('Total primitive signals:',len(signals))


# --- MODIFICATION (Phase 2, Item 4): Relaxing 3-Signal Restriction & Initial Filtering ---
# Replaced random sampling with combinatorial generation and support filter.
primitive_names=list(signals.keys())
all_candidate_setups = []
setup_id_counter = 1

print(f'\nGenerating setups with lengths {SETUP_LENGTHS_TO_EXPLORE} and applying initial support filter (min_support={MIN_INITIAL_SUPPORT_FILTER}) …')

for k in SETUP_LENGTHS_TO_EXPLORE:
    # Ensure there are enough primitive names for the current combination length
    if k > len(primitive_names):
        print(f"Warning: Cannot generate setups of length {k} as only {len(primitive_names)} primitives are available. Skipping.")
        continue

    for conds_tuple in itertools.combinations(primitive_names, k):
        conds_list = list(conds_tuple)

        # Calculate initial support for this combination
        try:
            # Ensure all signals for the combination actually exist
            valid_combo_signals = [signals[c] for c in conds_list if c in signals]
            if not valid_combo_signals: # Skip if no valid signals for this combination
                continue

            current_mask = functools.reduce(lambda a,b: a & b, valid_combo_signals)
            current_support = current_mask.sum()
        except KeyError:
            current_support = 0 # If any signal is missing, support is 0

        if current_support >= MIN_INITIAL_SUPPORT_FILTER:
            setup = {
                'id': 'S' + str(setup_id_counter).zfill(4),
                'conds': conds_list,
                'setup_length': k,
                'setup_type': 'concurrent', # Default for now, 'sequential' to be added later
                'support': current_support # Store initial support
            }
            all_candidate_setups.append(setup)
            setup_id_counter += 1

setups = all_candidate_setups # Use this filtered list for the full evaluation loop
print(f'Generated and filtered {len(setups)} candidate setups after initial support filter.')
# --- END MODIFICATION ---


# Prepare returns for evaluation (already updated in previous block)
# price_cols_for_returns correctly defined from TRADABLE_TICKERS earlier
# prices and returns dictionaries are already based on price_cols_for_returns
# so this section remains as is.

summary_rows=[]
trigger_records=[]

for setup in setups:
    sid=setup['id']; conds=setup['conds']

    valid_signals = [signals[c] for c in conds if c in signals]
    if not valid_signals:
        continue

    mask=functools.reduce(lambda a,b: a & b, valid_signals)
    dates=mask[mask].index
    support=len(dates)

    # We already filtered by MIN_INITIAL_SUPPORT_FILTER earlier, but keep this for sanity
    if support<5: # Can set this to MIN_INITIAL_SUPPORT_FILTER if desired for final check
        continue

    # --- NEW ADDITION (Phase 2, Item 2): Infer Directional Labels and Type ---
    direction_score = 0
    signal_type_counts = {}
    for cond in conds:
        direction_score += get_primitive_direction(cond)
        s_type = get_primitive_signal_type(cond)
        signal_type_counts[s_type] = signal_type_counts.get(s_type, 0) + 1

    entry_direction = 'mixed'
    if direction_score > 0: # Threshold for 'long' bias (e.g., >0 indicates more long signals)
        entry_direction = 'long'
    elif direction_score < 0: # Threshold for 'short' bias
        entry_direction = 'short'

    dominant_signal_type = 'unknown'
    if signal_type_counts:
        dominant_signal_type = max(signal_type_counts, key=signal_type_counts.get)

    # --- NEW ADDITION (Phase 2, Item 5): Basic Setup Lifecycle Tracking fields ---
    first_trigger_date = dates.min() if not dates.empty else pd.NaT
    last_trigger_date = dates.max() if not dates.empty else pd.NaT

    perf={'setup_id':sid,'feature_conditions':'+'.join(conds),'support':support,
          'entry_direction': entry_direction, # NEW
          'dominant_signal_type': dominant_signal_type, # NEW
          'first_trigger_date': first_trigger_date, # NEW
          'last_trigger_date': last_trigger_date # NEW
         }
    # --- END NEW ADDITION ---

    for h,label in zip([3,5,10,21],['accuracy_3d','avg_return_5d','sharpe_10d','hit_rate_21d']):
        r=returns[h].loc[dates]

        if r.empty:
            perf[label] = 0.0
            continue

        mean=r.mean(axis=1) # The mean return across all TRADABLE assets for each trigger date.

        mean = mean.dropna()
        if mean.empty:
            perf[label] = 0.0
            continue

        dir_correct=(mean>0).mean()
        if 'accuracy' in label:
            perf[label]=dir_correct
        elif 'avg' in label:
            perf[label]=mean.mean()
        elif 'sharpe' in label:
            bootstrap_block_size = min(h if h > 1 else 2, max(1, len(mean) // 2))

            # --- MODIFICATION (Phase 1, Item 1.2): Refined Sharpe Threshold ---
            # Penalize setups with extremely low standard deviation of mean returns
            if mean.std() > 0.0001 and len(mean) >= 2: # Increased threshold to 0.01% daily std dev
                median_sharpe, _, _ = block_bootstrap_sharpe(mean, block_size=bootstrap_block_size)
                perf[label] = median_sharpe
            else:
                perf[label] = 0.0 # Assign 0 if std dev is too small (meaning unrealistically stable) or not enough data
            # --- END MODIFICATION ---
        elif 'hit' in label:
            perf[label]=(r>0).stack().mean()
    summary_rows.append(perf)

    # Log individual ticker returns and NEW option PnL for triggered dates - ONLY TRADABLE TICKERS
    for d_idx, d in enumerate(dates): # Iterate with index to align with future price lookup
        for tk_col_full_name in price_cols_for_returns:
            tk_symbol_for_log = tk_col_full_name.split('_PX_LAST')[0].split('_Last_Price')[0]

            # Get current price and IVOL for option simulation
            current_px = safe_series(tk_col_full_name).loc[d]
            # Try to get 10-day call IVOL, or 30-day, or default to some other IVOL.
            # Use the most appropriate IVOL for the OPTION_SIM_HORIZON_DAYS
            ivol_col_for_sim = first_col_containing(tk_symbol_for_log, f'{OPTION_SIM_HORIZON_DAYS}_Day_Call_Implied_Volatility_CALL_IMP_VOL_{OPTION_SIM_HORIZON_DAYS}D')
            if not ivol_col_for_sim: # Fallback to 30D or 60D if 10D not found
                ivol_col_for_sim = first_col_containing(tk_symbol_for_log, '30_Day_Call_Implied_Volatility_CALL_IMP_VOL_30D')
            if not ivol_col_for_sim:
                ivol_col_for_sim = first_col_containing(tk_symbol_for_log, '60_Day_Call_Implied_Volatility_CALL_IMP_VOL_60D')

            ivol_at_entry = safe_series(ivol_col_for_sim).loc[d] if ivol_col_for_sim else np.nan

            option_pnl_10d = np.nan # Default to NaN
            # Ensure we have valid current price and IVOL before attempting option simulation
            if not pd.isna(current_px) and not pd.isna(ivol_at_entry):
                # Find future price for option simulation horizon
                future_date_for_sim = d + pd.Timedelta(days=OPTION_SIM_HORIZON_DAYS)
                # Find the actual row in raw for future_date_for_sim
                future_px_row = raw[raw['Date'] == future_date_for_sim]

                if not future_px_row.empty and tk_col_full_name in future_px_row.columns:
                    future_px_for_sim = future_px_row[tk_col_full_name].iloc[0]
                    # --- NEW ADDITION (Phase 2, Item 3): Simulate Option PnL ---
                    option_pnl_10d = simulate_option_pnl(current_px, future_px_for_sim, ivol_at_entry, OPTION_SIM_HORIZON_DAYS, entry_direction)
                    # --- END NEW ADDITION ---

            ret_vals={h:returns[h].loc[d,tk_col_full_name] if tk_col_full_name in returns[h].columns else np.nan for h in [1,3,5,10,21]}
            trigger_records.append({'date':d,'ticker':tk_symbol_for_log,'setup_id':sid,'matched':1,
                                    'return_1d':ret_vals[1],'return_3d':ret_vals[3],'return_5d':ret_vals[5],
                                    'return_10d':ret_vals[10],'return_21d':ret_vals[21],
                                    'option_pnl_10d': option_pnl_10d}) # NEW: Option PnL

summary_df=pd.DataFrame(summary_rows)

# --- NEW ADDITION (Phase 2, Item 5): Basic Setup Lifecycle Tracking Derived Metrics ---
if not summary_df.empty:
    summary_df['first_trigger_date'] = pd.to_datetime(summary_df['first_trigger_date'])
    summary_df['last_trigger_date'] = pd.to_datetime(summary_df['last_trigger_date'])

    # Calculate duration a setup was "active" from its first to last trigger
    summary_df['setup_duration_days'] = (summary_df['last_trigger_date'] - summary_df['first_trigger_date']).dt.days

    # Calculate frequency within its active duration, or against total data span
    # Replace 0 duration with NaN to avoid division by zero
    summary_df['avg_trigger_frequency_per_day'] = summary_df['support'] / summary_df['setup_duration_days'].replace(0, np.nan)
    # Replace inf with NaN for single-day triggers where duration is 0
    summary_df.replace([np.inf, -np.inf], np.nan, inplace=True)
else:
    print("Summary DataFrame is empty, skipping lifecycle metrics calculation.")
# --- END NEW ADDITION ---

top = summary_df[summary_df['sharpe_10d'] != 0.0].sort_values('sharpe_10d',ascending=False).head(20)

summary_df.to_csv('setup_results_summary.csv',index=False)
trigger_df=pd.DataFrame(trigger_records) # Ensure trigger_df is created from the collected records
trigger_df.to_csv('setup_trigger_log.csv',index=False)
top.to_json('top_setups.json',orient='records',indent=2)

print('\nDiscovery complete with engineered features')
print(top.head())

Loading raw workbooks …
An unexpected error occurred during Excel loading of 'Macro_tickers_no_nan_cols.xlsx': 'Date'
Raw shape: (1173, 404)
Example columns after loading:
['Date', 'QQQ US Equity_Last_Price_PX_LAST', 'QQQ US Equity_Open_Price_PX_OPEN', 'QQQ US Equity_High_Price_PX_HIGH', 'QQQ US Equity_Low_Price_PX_LOW']

Identified all relevant prefixes/tickers for feature engineering: ['AAPL US Equity', 'ARKK US Equity', 'CO1 Comdty', 'DXY Curncy', 'GLD US Equity', 'LLY US Equity', 'NBIS US Equity', 'NVDA US Equity', 'QQQ US Equity', 'SPY US Equity', 'TSLA US Equity', 'USGG10YR Index', 'VIX Index', 'XLE US Equity', 'XLF US Equity', 'XLK US Equity']
Actual tradable tickers for returns: ['QQQ US Equity', 'SPY US Equity', 'XLK US Equity', 'XLF US Equity', 'XLE US Equity', 'ARKK US Equity', 'VIX Index', 'GLD US Equity', 'NBIS US Equity', 'LLY US Equity', 'TSLA US Equity', 'AAPL US Equity', 'NVDA US Equity']

Engineering custom features …
  - Volatility-based features
  - Deriv Flow & Sen

KeyboardInterrupt: 